<a href="https://colab.research.google.com/github/UdayaPrakashST/BankruptcyModel_SHAP_LLM/blob/main/American_Bankruptcy_Prediction_1_Creating_the_modelling_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/UdayaPrakashST/BankruptcyModel_SHAP_LLM

Cloning into 'BankruptcyModel_SHAP_LLM'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 4.43 MiB | 12.92 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/BankruptcyModel_SHAP_LLM/american_bankruptcy.csv')

In [5]:
variable_mapping = {
    "X1": "currentAssets",
    "X2": "costOfGoodsSold",
    "X3": "depreciationAmortization",
    "X4": "ebitda",
    "X5": "inventory",
    "X6": "netIncome",
    "X7": "totalReceivables",
    "X8": "marketValue",
    "X9": "netSales",
    "X10": "totalAssets",
    "X11": "totalLongTermDebt",
    "X12": "ebit",
    "X13": "grossProfit",
    "X14": "totalCurrentLiabilities",
    "X15": "retainedEarnings",
    "X16": "totalRevenue",
    "X17": "totalLiabilities",
    "X18": "totalOperatingExpenses"
}

In [6]:
for i in variable_mapping.keys():
  df.rename(columns = {i:variable_mapping[i]},inplace=True)

In [7]:
df.head()

,company_name,status_label,year,currentAssets,costOfGoodsSold,depreciationAmortization,ebitda,inventory,netIncome,totalReceivables,...,netSales,totalAssets,totalLongTermDebt,ebit,grossProfit,totalCurrentLiabilities,retainedEarnings,totalRevenue,totalLiabilities,totalOperatingExpenses
0,C_1,alive,1999,511.267,833.107,18.373,89.031,336.018,35.163,128.348,...,1024.333,740.998,180.447,70.658,191.226,163.816,201.026,1024.333,401.483,935.302
1,C_1,alive,2000,485.856,713.811,18.577,64.367,320.590,18.531,115.187,...,874.255,701.854,179.987,45.790,160.444,125.392,204.065,874.255,361.642,809.888
2,C_1,alive,2001,436.656,526.477,22.496,27.207,286.588,-58.939,77.528,...,638.721,710.199,217.699,4.711,112.244,150.464,139.603,638.721,399.964,611.514
3,C_1,alive,2002,396.412,496.747,27.172,30.745,259.954,-12.410,66.322,...,606.337,686.621,164.658,3.573,109.590,203.575,124.106,606.337,391.633,575.592
4,C_1,alive,2003,432.204,523.302,26.680,47.491,247.245,3.504,104.661,...,651.958,709.292,248.666,20.811,128.656,131.261,131.884,651.958,407.608,604.467


In [10]:
df.groupby(['year'],as_index=False).agg({'company_name':'nunique'})

,year,company_name
0,1999,5308
1,2000,5226
2,2001,4897
3,2002,4651
4,2003,4417
5,2004,4348
6,2005,4205
7,2006,4128
8,2007,4009
9,2008,3857


In [11]:
df['status_label'].value_counts()

,count
status_label,
alive,73462
failed,5220


In [14]:
company_failures = df[df['status_label']=='failed'].groupby(['company_name'],as_index=False).agg({'year':'max'})
company_failures.head()

,company_name,year
0,C_1020,2004
1,C_1046,2017
2,C_1058,2014
3,C_1085,2004
4,C_1114,2007


In [13]:
company_failures['year'].value_counts().sort_index()

,count
year,
1999,3
2000,7
2001,10
2002,17
2003,29
2004,46
2005,40
2006,51
2007,59


In [15]:
df_failures = pd.merge(df,
company_failures,
how='inner',on = ['company_name','year'])

In [16]:
df_failures['bad_ind'] = 1

In [17]:
df_combined = pd.merge(df,
                       df_failures[['company_name','year','bad_ind']],
                       how='left',
                       on = ['company_name','year'])
df_combined['bad_ind'].fillna(0,inplace=True)

<ipython-input-17-9a04eb271594>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_combined['bad_ind'].fillna(0,inplace=True)


In [18]:
df_combined['bad_ind'].value_counts()

,count
bad_ind,
0.0,78073
1.0,609


In [20]:
df_combined.groupby(['year'],as_index=False).agg({'company_name':'count','bad_ind':'sum'}).rename(columns = {'company_name':'Records',
                                                                                                             'bad_ind':'Bankruptcies'}).eval('bkcy_rate = Bankruptcies/Records')

,year,Records,Bankruptcies,bkcy_rate
0,1999,5308,3.0,0.000565
1,2000,5226,7.0,0.001339
2,2001,4897,10.0,0.002042
3,2002,4651,17.0,0.003655
4,2003,4417,29.0,0.006566
5,2004,4348,46.0,0.010580
6,2005,4205,40.0,0.009512
7,2006,4128,51.0,0.012355
8,2007,4009,59.0,0.014717
9,2008,3857,58.0,0.015038


In [21]:
df_combined.to_csv('/content/BankruptcyModel_SHAP_LLM/bankruptcy_model_dataset.csv',index=False)